In [1]:
__PRODUCTION__ = 1
__NAME__       = 'er-analysis'
__WIDTH__      = 5.5  # NeurIPS 2021 text box width
__HEIGHT__     = 1.55

if __PRODUCTION__:
    from mplmagic2 import pgf
else:
    from mplmagic2 import svg

from mplmagic2 import SuperFigure
import matplotlib.pyplot as plt

print('This is how much space the figure will take up on letter paper')
SuperFigure.size_hint(__WIDTH__, __HEIGHT__, margin_left=0.5 * (8.5 - __WIDTH__));

This is how much space the figure will take up on letter paper


In [2]:
import functools
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.distance import cdist
import torch
import torch.nn.functional as F
from torch import optim
import networkx as nx
import pickle
from sklearn import metrics

from symfac.experimental import RBFExpansionV2

In [3]:
svd2_approx, rbf2_approx,K00=pickle.load(open('data/paper/er_approx_2.pickle', "rb"))
fpr2, tpr2, fpr_svd2, tpr_svd2 = pickle.load(open('data/paper/er_auc_2.pickle', "rb"))
fpr7, tpr7, fpr_svd7, tpr_svd7 = pickle.load(open('data/paper/er_auc_7.pickle', "rb"))

In [4]:
fig = SuperFigure(plt.figure(figsize=(__WIDTH__, __HEIGHT__), dpi=300))
ax_canvas = fig.make_axes(
    left=0, right=1, top=0, bottom=1, zorder=-100,
    style='blank' if __PRODUCTION__ else None
)
ax_canvas.set_xlim([0, 1])
ax_canvas.set_ylim([0, 1])

# draw the axes grid
# x0 = 0.05
# dx = 0.24
# w = 0.21
x0 = 0.005
dx = 0.245
w = 0.23
axs = [
    fig.make_axes(
        left=x0 + dx * i,
        width=w,
        top=0.09,
        width_to_height=1.0,
        style='modern'
    ) for i in range(4)
]
bbox = axs[3].get_position()
bbox.x0 += 0.02
bbox.x1 += 0.02
axs[3].set_position(bbox)

image_style = dict(
    vmin=0,
    vmax=1,
    cmap='Greys_r'
)
axs[0].imshow(K00, **image_style)
axs[1].imshow(rbf2_approx, **image_style)
axs[2].imshow(svd2_approx, **image_style)

svd_base_style = dict(
    lw=0.75,
    dash_capstyle='round'
)
svd2_style = dict(
    ls=(2, (1.25, 2.5)),
    **svd_base_style
)
svd7_style = dict(
    ls=(4, (0.5, 2.5, 3, 2.5)),
    **svd_base_style
)
rbf_base_style = dict(
    lw=1.0,
    markersize=3,
    markeredgewidth=0.75,
    markerfacecolor='w',
)
rbf2_style = dict(
    marker='o',
    markevery=30,
    **rbf_base_style
)
rbf7_style = dict(
    marker='^',
    markevery=5,
    **rbf_base_style
)
lo_style = dict(
    color='#F06060'
)
hi_style = dict(
    color='#606060'
)
axs[3].plot(fpr_svd2, tpr_svd2, label='SVD (2), AUC = 0.72',  **svd7_style, **lo_style)
axs[3].plot(fpr2, tpr2, label='RBF (2), AUC = 0.82', **rbf2_style, **lo_style)
axs[3].plot(fpr_svd7, tpr_svd7, label='SVD (7), AUC = 0.90',  **svd2_style, **hi_style)
axs[3].plot(fpr7, tpr7, label='RBF (7), AUC = 0.98',  **rbf7_style, **hi_style)

title_style = dict(
    fontsize=7,
    y=1.02,
    va='top'
)
tick_style = dict(
    fontsize=7,
)
label_style = dict(
    fontsize=8,
    labelpad=0.75,
)


for i in range(4):
    if i != 3:
        ticks = [0, 40]
        axs[i].set_xbound(lower=0, upper=40)
        axs[i].set_ybound(lower=0, upper=40)
    else:
        ticks = [0, 1]
        axs[i].set_xbound(lower=-0.02, upper=1.02)
        axs[i].set_ybound(lower=-0.02, upper=1.02)
    axs[i].set_xticks(ticks)
    axs[i].set_yticks(ticks)
    axs[i].set_xticklabels(['%d' % x for x in ticks], **tick_style)
    if i == 3:
        axs[i].set_yticklabels(['%d' % y for y in ticks], **tick_style)
    else:
        axs[i].set_yticklabels([])

axs[0].set_title(r'\textbf{Adjacency matrix}', **title_style)
axs[1].set_title(r'\textbf{2-Component RBF}', **title_style)
axs[2].set_title(r'\textbf{2-Rank SVD}', **title_style)
axs[3].set_title(r'\textbf{AUC-ROC}', **title_style)
axs[3].legend(
    loc='lower right',
    fontsize=7,
    frameon=False
)


if __PRODUCTION__:
    fig.savefig(f'pgf/{__NAME__}.pgf', dpi=300)
else:
    fig.savefig(f'svg/{__NAME__}.svg', dpi=300)
plt.show()

In [5]:
!make -f Makefile.figures fig-"$__NAME__".pdf 2>&1 | tail -n 1

Successfully created fig-er-analysis.pdf


In [6]:
end

NameError: name 'end' is not defined

# END OF PLOTTING

In [ ]:
K00 = pickle.loads(open('data/er_graph.pickle', 'rb').read())


In [ ]:
def threshold(alpha, array):
    tresholded_array = torch.randn(array.shape)
    tresholded_array[array < alpha] = 0
    tresholded_array[array >= alpha] = 1
    return tresholded_array

In [ ]:
errors = []
bs=100
for k in range(2,3):
    u0=1/1000*torch.randn([bs, len(K00), k]) 
    a=torch.randn([bs, k], device="cuda")
    b=torch.randn([bs], device="cuda")
    fac = RBFExpansionV2(k=k, batch_size=bs, max_steps=40000)
    fac.fith(
        K00,
        u0=u0,
        a0=a,
        b0=b
    )
    plt.loglog(
        fac.report.loss_history_ticks,
        fac.report.loss_history.min(axis=1)
    )
    plt.show()

    error = fac.report.loss_best
    er = round(float(torch.min(error).cpu().detach().numpy()), 7)
    print(er)
    errors.append(er)
    plt.imshow(fac.optimum(runs=torch.argmin(fac.report.loss_best)))
    plt.show()
print(errors)

In [ ]:
plt.loglog(
        fac.report.loss_history_ticks,
        fac.report.loss_history.min(axis=1)
    )
plt.show()

error = fac.report.loss_best
er = round(float(torch.min(error).cpu().detach().numpy()), 7)
print(er)
errors.append(er)
plt.imshow(fac.optimum(runs=torch.argmin(fac.report.loss_best)))
plt.show()

In [ ]:
rank2_approx = fac.optimum(runs=torch.argmin(fac.report.loss_best))

In [ ]:
U, S, V = np.linalg.svd(K00.detach().numpy())
svd2_approx = torch.tensor((U[:, :2] * S[None, :2]) @ V[:2, :])

In [ ]:
from sklearn import metrics
y = K00.numpy().flatten()
pred = rank2_approx.numpy().flatten()
pred_svd = svd2_approx.numpy().flatten()

fpr2, tpr2, thresholds = metrics.roc_curve(y, pred)
print(metrics.auc(fpr2, tpr2))
fpr_svd2, tpr_svd2, thresholds2 = metrics.roc_curve(y, pred_svd)
print(metrics.auc(fpr_svd2, tpr_svd2))

In [ ]:
plt.plot(fpr2, tpr2, label='Two components RBF, AUC=0.82',  linewidth=3, linestyle='-')
plt.plot(fpr_svd2, tpr_svd2, label='Two ranks SVD, AUC=0.72',  linewidth=3, linestyle=':')
plt.legend(loc='lower right', fontsize=12)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.show


In [ ]:
open('data/er_auc_2.pickle', 'wb').write(pickle.dumps([fpr2, tpr2, fpr_svd2, tpr_svd2]))

In [ ]:
open('data/er_rbf2_approx.pickle', 'wb').write(pickle.dumps(fac.optimum(runs=torch.argmin(fac.report.loss_best))))

In [ ]:
errors7 = []
bs=100
for k in range(7,8):
    u0=1/1000*torch.randn([bs, len(K00), k]) 
    a=torch.randn([bs, k], device="cuda")
    b=torch.randn([bs], device="cuda")
    fac = RBFExpansionV2(k=k, batch_size=bs, max_steps=40000)
    fac.fith(
        K00,
        u0=u0,
        a0=a,
        b0=b
    )
    plt.loglog(
        fac.report.loss_history_ticks,
        fac.report.loss_history.min(axis=1)
    )
    plt.show()

    plt.show()
    error = fac.report.loss_best
    er = round(float(torch.min(error).cpu().detach().numpy()), 7)
    print(er)
    errors7.append(er)
    plt.imshow(fac.optimum(runs=torch.argmin(fac.report.loss_best)))
    plt.show()
    #open('er_{0}.pickle'.format(k), 'wb').write(pickle.dumps(fac.optimum.x))
print(errors7)

In [ ]:
rank7_approx = fac.optimum(runs=torch.argmin(fac.report.loss_best))

In [ ]:
U, S, V = np.linalg.svd(K00.detach().numpy())
svd7_approx = torch.tensor((U[:, :7] * S[None, :7]) @ V[:7, :])

In [ ]:
pred_7 = rank7_approx.numpy().flatten()
pred_svd_7 = svd7_approx.numpy().flatten()

fpr7, tpr7, thresholds = metrics.roc_curve(y, pred_7)
print(metrics.auc(fpr, tpr))
fpr_svd7, tpr_svd7, thresholds_svd7 = metrics.roc_curve(y, pred_svd_7)
print(metrics.auc(fpr_svd7, tpr_svd7))

In [ ]:
plt.plot(fpr7, tpr7, label='7 components RBF, AUC=0.98',  linewidth=3, linestyle='-')
plt.plot(fpr_svd7, tpr_svd7, label='7 ranks SVD, AUC=0.9',  linewidth=3, linestyle=':')
plt.legend(loc='lower right', fontsize=12)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.show

In [ ]:
open('data/er_auc_7.pickle', 'wb').write(pickle.dumps([fpr7, tpr7, fpr_svd7, tpr_svd7]))

In [ ]:
fig = SuperFigure(plt.figure(figsize=(__WIDTH__, __HEIGHT__)))
ax_canvas = fig.make_axes(
    left=0, right=1, top=0, bottom=1, zorder=-100,
    style='blank' if __PRODUCTION__ else None
)
ax_canvas.set_xlim([0, 1])
ax_canvas.set_ylim([0, 1])

# draw the axes grid
x0 = 0.05
dx = 0.24
w = 0.2
axs = [
    fig.make_axes(
        left=x0 + dx * i,
        width=w,
        top=0.1,
        width_to_height=1.0,
        style='modern'
    ) for i in range(3)
]

axs[1].imshow(fac.optimum(runs=torch.argmin(fac.report.loss_best)))
axs[0].imshow(K00, cmap='hot', interpolation='nearest')
axs[2].imshow(svd2_approx)


if __PRODUCTION__:
    fig.savefig(f'pgf/{__NAME__}.pgf')
else:
    fig.savefig(f'svg/{__NAME__}.svg')
plt.show()

In [ ]:
open('data/er_approx_2.pickle', 'wb').write(pickle.dumps([svd2_approx, fac.optimum(runs=torch.argmin(fac.report.loss_best)), K00]))

In [ ]:
 plt.imshow(fac.optimum(runs=torch.argmin(fac.report.loss_best)))

In [ ]:
 plt.imshow(svd2_approx)

In [ ]:
plt.imshow(K00, cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
svd1_approx = torch.tensor((U0[:, :1] * S0[None, :1]) @ V0[:1, :])

In [ ]:
round(float(torch.min(error).cpu().detach().numpy()), 7)

In [ ]:
fig = plt.figure(figsize=plt.figaspect(1))
#===============
# set up the axes for the second plot
ax = fig.add_subplot(1, 1, 1)

ax.plot(fpr2, tpr2, label='Two components RBF, AUC=0.81',  linewidth=3, linestyle='-')
ax.plot(fpr_svd2, tpr_svd2, label='Two ranks SVD, AUC=0.72',  linewidth=3, linestyle=':')
ax.plot(fpr7, tpr7, label='Seven components RBF, AUC=0.99',  linewidth=3, linestyle='-')
ax.plot(fpr_svd7, tpr_svd7, label='Seven ranks SVD, AUC=0.9',  linewidth=3, linestyle=':')
ax.legend(loc='lower right', fontsize=9)


plt.show()

In [ ]:
fpr2, tpr2, fpr_svd2, tpr_svd2 = pickle.load(open('er_auc_2.pickle', "rb"))
fpr7, tpr7, fpr_svd7, tpr_svd7 = pickle.load(open('er_auc_7.pickle', "rb"))

In [ ]:
tick_style = dict(
    fontsize=7,
)
label_style = dict(
    fontsize=8,
    labelpad=0.75,
)
for i, ax in enumerate(axs):
    ticks = [0, 1]
    ax.set_xbound(lower=0, upper=1)
    ax.set_ybound(lower=0, upper=1)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(['%d' % x for x in ticks], **tick_style)
    if i == 0:
        ax.set_yticklabels(['%d' % y for y in ticks], **tick_style)
        ax.set_ylabel(r'\textbf{Learned embedding}', **label_style)
    else:
        ax.set_yticklabels([])
    ax.set_xlabel(r'\textbf{Ground truth}', **label_style)